In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

df = pd.read_csv(r"C:\Users\umber\Desktop\aiutoo.csv")
def replace_case(value):
    if value <= 105:
        return 0
    else:
        return 1

df['case'] = df['case'].apply(replace_case)
df = df.fillna(0)

X_smote = df[["P1","P2","P3","P4","P5","P6","P7","P1_FFT","P2_FFT","P3_FFT","P4_FFT","P5_FFT","P6_FFT","P7_FFT"]]
y_smote = df["case"]

x_train_sv, x_test_sv, y_train_sv, y_test_sv = train_test_split(X_smote, y_smote, test_size=0.20)
model = RandomForestClassifier(max_depth = None, min_samples_leaf = 1, min_samples_split = 2, n_estimators = 200, random_state=42)
model.fit(x_train_sv, y_train_sv)
predictions = model.predict(x_test_sv)

print(classification_report(y_test_sv, predictions))

              precision    recall  f1-score   support

           0       0.99      0.95      0.97     25128
           1       0.93      0.98      0.96     17388

    accuracy                           0.96     42516
   macro avg       0.96      0.97      0.96     42516
weighted avg       0.96      0.96      0.96     42516



In [2]:
from sklearn.model_selection import cross_val_score


rf_classifier = RandomForestClassifier()

# Esegui la cross-validation con 5 fold
scores = cross_val_score(rf_classifier, X_smote, y_smote, cv=5)

# Stampiamo i punteggi ottenuti dalla cross-validation
print("Punteggi della cross-validation:", scores)

# Calcola la media dei punteggi
print("Media dei punteggi:", scores.mean())

Punteggi della cross-validation: [0.99769499 0.99550757 0.91127837 0.96083735 0.54799483]
Media dei punteggi: 0.8826626216509043


In [11]:
# creazione tabella frequenza
import numpy as np
import pywt
from sklearn.discriminant_analysis import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from scipy.fft import rfft
from scipy.fft import fft

from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
import pandas as pd
from imblearn.over_sampling import SMOTE



from sklearn.metrics import classification_report
df = pd.read_csv(r"C:\Users\umber\Desktop\MANUTENZIONE\dataset_test_finale.csv")
df = pd.concat([df]*2, ignore_index=True)

df = df[["Case","P1","P2","P3","P4","P5","P6","P7"]]
#ALLENAMENTO MODELLO
X=df[["P1","P2","P3","P4","P5","P6","P7"]]

df_train = pd.DataFrame()
peaks_train = {}


for colonna in X.columns:
    df_train[colonna+"_FFT"] = rfft(X[colonna].values)



num_rows = len(df_train)
case_values = []
current_case = 1
for i in range(num_rows):
    case_values.append(current_case)
    if (i + 1) % 1201 == 0:
        current_case += 1


# Aggiungi la colonna "case" al dataframe
df_train['case'] = case_values



last_col_index = len(df_train.columns) - 1

for colonna in df_train.columns[:last_col_index]:
    peaks_train[colonna+"peaks"] = find_peaks(df_train[colonna], threshold= 1, distance=10) #7 elementi con le posizioni

for colonna in df_train.columns[:last_col_index]:
    df_train[colonna] = df_train[colonna][peaks_train[colonna+"peaks"][0]]

df_train = df_train.fillna(0)
df_train = df_train.applymap(lambda x: np.real(x))
def replace_case(value):
    if value <= 105:
        return 0
    else:
        return 1

#df_train['case'] = df_train['case'].apply(replace_case)
df_train = df_train[(df_train.iloc[:, :7] != 0).any(axis=1)]

X = df_train [["P1_FFT","P2_FFT","P3_FFT","P4_FFT","P5_FFT","P6_FFT","P7_FFT"]]
y = df_train["case"]

smote = SMOTE()
X_smote, y_smote = smote.fit_resample(X, y)

df_smote = pd.concat([pd.DataFrame(X_smote, columns=X.columns), pd.DataFrame(y_smote, columns=["case"])], axis=1)
df_smote



c:\Users\umber\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\series.py:917: ComplexWarning: Casting complex values to real discards the imaginary part
  arr = np.asarray(values, dtype=dtype)


,P1_FFT,P2_FFT,P3_FFT,P4_FFT,P5_FFT,P6_FFT,P7_FFT,case
0,743.392862,7.569112e+02,735.034666,834.733228,744.083713,756.635518,731.041716,1
1,6902.791604,6.905400e+03,6907.086661,6889.113261,6908.949793,6981.118700,6912.588340,1
2,2218.745091,2.210687e+03,2214.839820,2120.135751,2202.331865,2191.570972,2211.086383,1
3,3130.031337,3.122657e+03,3124.602335,3090.670845,3123.866207,3134.170611,3136.645217,1
4,1873.104888,1.872899e+03,1886.358642,1840.613666,1872.588454,1871.257311,1870.411220,1
...,...,...,...,...,...,...,...,...
19223,0.000000,1.338196e+01,0.000000,0.000000,56.579902,0.000000,0.000000,46
19224,0.000000,-5.866231e-14,0.000000,0.000000,0.000000,0.000000,0.000000,46
19225,0.000000,0.000000e+00,0.000000,1.060953,0.000000,0.000000,0.000000,46
19226,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,51.697559,46


In [12]:
import pandas as pd

# Supponiamo che il tuo DataFrame si chiami df e abbia una colonna chiamata 'case'

# Creiamo un DataFrame vuoto
new_df = pd.DataFrame()

# Per ogni valore univoco nella colonna 'case'
for value in df_smote['case'].unique():
    # Estraiamo solo le righe con quel valore nella colonna 'case'
    subset = df_smote[df_smote['case'] == value]
    
    # Calcoliamo quante righe mancano per arrivare a 1201
    missing_rows = 1201 - len(subset)
    
    # Se mancano righe, aggiungiamo righe con zeri
    if missing_rows > 0:
        # Creiamo un DataFrame temporaneo con righe di zeri
        temp_df = pd.DataFrame([[value] + [0] * (len(df_smote.columns) - 1)] * missing_rows, columns=df_smote.columns)
        
        # Aggiungiamo il DataFrame temporaneo al nuovo DataFrame
        new_df = pd.concat([new_df, subset, temp_df], ignore_index=True)
    else:
        # Se il numero di righe è già uguale a 1201, aggiungiamo semplicemente il subset al nuovo DataFrame
        new_df = pd.concat([new_df, subset], ignore_index=True)

new_df

,P1_FFT,P2_FFT,P3_FFT,P4_FFT,P5_FFT,P6_FFT,P7_FFT,case
0,743.392862,756.911219,735.034666,834.733228,744.083713,756.635518,731.041716,1
1,6902.791604,6905.400075,6907.086661,6889.113261,6908.949793,6981.118700,6912.588340,1
2,2218.745091,2210.686568,2214.839820,2120.135751,2202.331865,2191.570972,2211.086383,1
3,3130.031337,3122.657022,3124.602335,3090.670845,3123.866207,3134.170611,3136.645217,1
4,1873.104888,1872.899348,1886.358642,1840.613666,1872.588454,1871.257311,1870.411220,1
...,...,...,...,...,...,...,...,...
55241,46.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
55242,46.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
55243,46.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
55244,46.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0


In [13]:
#creazione tabella tempo
df = pd.read_csv(r"C:\Users\umber\Desktop\MANUTENZIONE\dataset_test_finale.csv")
df_train1 = df[["P1","P2","P3","P4","P5","P6","P7","Case"]]
def replace_case(value):
    if value <= 105:
        return 0
    else:
        return 1

#df_train1['Case'] = df_train1['Case'].apply(replace_case)
df_train1.rename(columns={'Case': 'case'}, inplace=True)
df_train1




C:\Users\umber\AppData\Local\Temp\ipykernel_46428\2580221331.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train1.rename(columns={'Case': 'case'}, inplace=True)


,P1,P2,P3,P4,P5,P6,P7,case
0,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,178
1,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,178
2,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,178
3,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,178
4,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,178
...,...,...,...,...,...,...,...,...
55241,2.017816,2.016297,2.012680,2.044149,2.037457,2.033628,2.069056,223
55242,2.039878,2.055107,2.054626,2.071298,2.072899,2.076782,2.093548,223
55243,2.054024,2.081678,2.094493,2.096693,2.095300,2.107353,2.138567,223
55244,2.057180,2.094954,2.097106,2.153193,2.099423,2.112326,2.127471,223


In [14]:
new_df = new_df[["P1_FFT","P2_FFT","P3_FFT","P4_FFT","P5_FFT","P6_FFT","P7_FFT"]]
df_train1 = df_train1.join(new_df)
df_train1


,P1,P2,P3,P4,P5,P6,P7,case,P1_FFT,P2_FFT,P3_FFT,P4_FFT,P5_FFT,P6_FFT,P7_FFT
0,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,178,743.392862,756.911219,735.034666,834.733228,744.083713,756.635518,731.041716
1,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,178,6902.791604,6905.400075,6907.086661,6889.113261,6908.949793,6981.118700,6912.588340
2,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,178,2218.745091,2210.686568,2214.839820,2120.135751,2202.331865,2191.570972,2211.086383
3,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,178,3130.031337,3122.657022,3124.602335,3090.670845,3123.866207,3134.170611,3136.645217
4,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,178,1873.104888,1872.899348,1886.358642,1840.613666,1872.588454,1871.257311,1870.411220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55241,2.017816,2.016297,2.012680,2.044149,2.037457,2.033628,2.069056,223,46.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
55242,2.039878,2.055107,2.054626,2.071298,2.072899,2.076782,2.093548,223,46.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
55243,2.054024,2.081678,2.094493,2.096693,2.095300,2.107353,2.138567,223,46.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
55244,2.057180,2.094954,2.097106,2.153193,2.099423,2.112326,2.127471,223,46.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
train = pd.read_csv(r"C:\Users\umber\Desktop\aiutoo.csv")
def replace_case(value):
    if value <= 105:
        return 0
    else:
        return 1

train['case'] = train['case'].apply(replace_case)
train = train.fillna(0)
x = train[["P1_FFT","P2_FFT","P3_FFT","P4_FFT","P5_FFT","P6_FFT","P7_FFT","P1","P2","P3","P4","P5","P6","P7"]]
y = train["case"]
test = df_train1 [["P1_FFT","P2_FFT","P3_FFT","P4_FFT","P5_FFT","P6_FFT","P7_FFT","P1","P2","P3","P4","P5","P6","P7"]]
colonna = df_train1["case"]
model = RandomForestClassifier(random_state=42)
model.fit(x, y)
predictions = model.predict(test)
test['Prediction'] = predictions
df_unito = pd.concat([colonna, test], axis=1)
grouped_data = df_unito.groupby('case')['Prediction'].apply(lambda x: x.mode().iat[0]).reset_index() # restituisce il valore più frequente (moda)
grouped_data

,case,Prediction
0,178,0
1,179,0
2,180,0
3,181,0
4,182,0
5,183,0
6,184,0
7,185,0
8,186,0
9,187,0
